In [1714]:
import pandas as pd 
import numpy as np 
import os 
import requests
import joblib
from pprint import pprint
import matplotlib.pyplot as plt
import seaborn as sns
import glob
from sklearn.preprocessing import LabelEncoder, StandardScaler, PolynomialFeatures, MinMaxScaler
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.impute import KNNImputer
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from scipy import stats 
from sklearn.cluster import KMeans
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from sklearn.cluster import KMeans
from sklearn.impute import SimpleImputer
from itertools import combinations
import random

In [1715]:
goalies_rec = pd.read_csv('/Users/blairjdaniel/lighthouse/lighthouse/NHL/files/master_copies/goalies_rec_two.csv')
goalies_quant = pd.read_csv('/Users/blairjdaniel/lighthouse/lighthouse/NHL/files/nhlapigoalies/QuantHockey.csv')
forwards_rec = pd.read_csv('/Users/blairjdaniel/lighthouse/lighthouse/NHL/files/master_copies/forwards_rec_two.csv')
defense_rec = pd.read_csv('/Users/blairjdaniel/lighthouse/lighthouse/NHL/files/master_copies/defense_rec_two.csv')

In [1716]:
defense_rec

,Unnamed: 0,name,d_zone_shift_starts,giveaways,goals,high_danger_goals,high_danger_shots,hits,low_danger_goals,low_danger_shots,...,games_played,icetime,on_ice_corsi_percentage,on_ice_fenwick_percentage,penalties_drawn,shots_blocked_by_player,assists,position_encoded,position,salary
0,0,Aaron Ekblad,3.949480,0.922734,0.169391,0.041605,0.104012,1.105498,0.092125,3.004458,...,673,1369.442793,0.008083,0.008024,0.169391,1.074294,0.340267,1,D,8000000
1,1,Aaron Johnson,2.171233,0.424658,0.068493,0.027397,0.047945,1.568493,0.041096,1.308219,...,146,935.164384,0.013562,0.013630,0.164384,1.068493,0.164384,1,D,950000
2,2,Aaron MacKenzie,2.000000,0.000000,0.000000,0.000000,0.000000,1.200000,0.000000,0.800000,...,5,878.000000,0.094000,0.104000,0.200000,1.600000,0.000000,1,D,950000
3,3,Aaron Ness,1.388889,0.319444,0.013889,0.000000,0.027778,0.583333,0.013889,1.069444,...,72,823.791667,0.045139,0.043750,0.125000,1.111111,0.083333,1,D,950000
4,4,Aaron Rome,2.697115,0.350962,0.024038,0.009615,0.019231,1.322115,0.009615,1.235577,...,208,934.129808,0.013942,0.013894,0.110577,1.192308,0.100962,1,D,950000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
969,969,Zach Werenski,3.500000,0.551440,0.185185,0.032922,0.104938,0.584362,0.104938,3.236626,...,486,1403.553498,0.008868,0.008827,0.092593,1.269547,0.436214,1,D,8000000
970,970,Zach Whitecloud,2.773279,0.360324,0.068826,0.008097,0.040486,1.668016,0.048583,1.502024,...,247,1055.174089,0.011781,0.012105,0.129555,1.655870,0.165992,1,D,6000000
971,971,Zachary Hayes,0.666667,1.000000,0.000000,0.000000,0.000000,4.000000,0.000000,0.666667,...,3,653.000000,0.176667,0.156667,0.000000,0.333333,0.000000,1,D,950000
972,972,Zbynek Michalek,4.892514,0.397313,0.046065,0.005758,0.017274,1.000000,0.030710,1.708253,...,521,1267.504798,0.007236,0.007313,0.147793,2.412668,0.159309,1,D,4000000


In [1717]:
# Look at the forwards df
defense_rec = defense_rec.drop(columns=['position_encoded'])
forwards_rec = forwards_rec.drop(columns=['position_encoded'])

In [1718]:
# Define the salary cap
salary_cap = 30000000

In [1719]:
# Define the team composition requirements
team_composition = {'F': 3, 'D': 2}
desired_team_count = sum(team_composition.values())

In [1720]:
performance_metrics = [
    'goals', 'high_danger_goals', 'high_danger_shots', 'hits',
    'penalty_minutes', 'points', 'shifts', 
    'shots_on_goal', 'games_played', 
    'assists', 
]

In [1721]:
# Scale the features
scaler = StandardScaler()
forwards_rec[performance_metrics] = scaler.fit_transform(forwards_rec[performance_metrics])
defense_rec[performance_metrics] = scaler.fit_transform(defense_rec[performance_metrics])
#goalies[performance_metrics_goalies] = scaler.fit_transform(goalies[performance_metrics_goalies])

In [1722]:
# Function to recommend the next player based on cosine similarity
def recommend_next_player_skater(selected_player, players_pool, remaining_cap, performance_metrics, reserve=0):
    # Filter candidates by position and salary
    pos_candidates = players_pool[(players_pool['salary'] <= remaining_cap - reserve)].copy()
    
    if pos_candidates.empty:
        return None, remaining_cap
    
    # Compute cosine similarity between the selected player and candidates
    pos_candidates['similarity'] = cosine_similarity(
        pos_candidates[performance_metrics], 
        selected_player[performance_metrics].values.reshape(1, -1)
    )[:, 0]
    
    # Select the candidate with the highest similarity
    best_candidate = pos_candidates.sort_values(by='similarity', ascending=False).iloc[0]
    remaining_cap -= best_candidate['salary']
    
    return best_candidate, remaining_cap

In [1723]:
# Example Usage
user_selected_name = "Ryan Kesler"  # Assuming the names are lowercased
selected_player = forwards_rec[forwards_rec['name'] == user_selected_name].iloc[0]

# Remove the selected player from the pool
forwards_rec = forwards_rec.drop(selected_player.name)
remaining_cap = salary_cap - selected_player['salary']

In [1724]:
# Initialize the team with the selected player
recommended_team = [selected_player]
print(f"Added {selected_player['name']} ({selected_player['position']}) - ${selected_player['salary']:,}")
print(f"Remaining Salary Cap: ${remaining_cap:,}")

# Adjust team composition based on the selected player's position
if selected_player['position'] == 'F':
    team_composition['F'] -= 1
else:
    team_composition['D'] -= 1

# Shuffle the positions to pick players from different positions in a random order
positions = ['F', 'D']
random.shuffle(positions)

# Recommend players for each position
for position in positions:
    count = team_composition[position]
    for _ in range(count):
        # Determine reserve: if this is not the final pick overall, reserve $1,000,000.
        if len(recommended_team) < desired_team_count - 1:
            reserve = 1000000
        else:
            reserve = 0
        
        if position == 'F':
            next_player, remaining_cap = recommend_next_player_skater(selected_player, forwards_rec, remaining_cap, performance_metrics, reserve)
            if next_player is not None:
                recommended_team.append(next_player)
                forwards_rec = forwards_rec.drop(next_player.name)
                print(f"Added {next_player['name']} ({next_player['position']}) - ${next_player['salary']:,}")
                print(f"Remaining Salary Cap: ${remaining_cap:,}")
            else:
                print(f"No suitable forward candidate found with at least $1,000,000 reserved.")
                break
        elif position == 'D':
            next_player, remaining_cap = recommend_next_player_skater(selected_player, defense_rec, remaining_cap, performance_metrics, reserve)
            if next_player is not None:
                recommended_team.append(next_player)
                defense_rec = defense_rec.drop(next_player.name)
                print(f"Added {next_player['name']} ({next_player['position']}) - ${next_player['salary']:,}")
                print(f"Remaining Salary Cap: ${remaining_cap:,}")
            else:
                print(f"No suitable defense candidate found with at least $1,000,000 reserved.")
                break

# Final check: must have exactly 5 players (3 F, 2 D)
if (len([p for p in recommended_team if p['position'] == 'F']) == 3 and
    len([p for p in recommended_team if p['position'] == 'D']) == 2):
    # Convert recommended team to a DataFrame to view/inspect the team
    recommended_team_df = pd.DataFrame(recommended_team)
    print("Recommended Team:")
    print(recommended_team_df[['name', 'position', 'salary', 'points', 'goals', 'assists']])
    print(f"Remaining Salary Cap: ${remaining_cap:,}")
else:
    print("Failed to meet the team composition requirements.")


Added Ryan Kesler (F) - $6,000,000
Remaining Salary Cap: $24,000,000
Added Mike Richards (F) - $6,000,000
Remaining Salary Cap: $18,000,000
Added Gabriel Landeskog (F) - $8,000,000
Remaining Salary Cap: $10,000,000
Added Oliver Ekman-Larsson (D) - $8,000,000
Remaining Salary Cap: $2,000,000
Added Bryan McCabe (D) - $950,000
Remaining Salary Cap: $1,050,000
Recommended Team:
                      name position   salary    points     goals   assists
1619           Ryan Kesler        F  6000000  1.413987  1.422228  1.171674
1317         Mike Richards        F  6000000  1.403232  1.040074  1.431250
651      Gabriel Landeskog        F  8000000  1.952182  1.835559  1.710186
725   Oliver Ekman-Larsson        D  8000000  1.463318  1.696593  1.061740
145           Bryan McCabe        D   950000  1.610692  1.535077  1.304267
Remaining Salary Cap: $1,050,000


In [1725]:
# # Initialize the team with the selected player
# recommended_team = [selected_player]

# # Shuffle the positions to pick players from different positions in a random order
# positions = ['F', 'D']
# random.shuffle(positions)

# # Recommend players for each position
# player_count = 1  # Start with the first selected player
# for position in positions:
#     count = team_composition[position]
#     for _ in range(count):
#         if position == 'F':
#             next_player, remaining_cap = recommend_next_player_skater(selected_player, forwards_rec, remaining_cap, performance_metrics)
#             if next_player is not None:
#                 recommended_team.append(next_player)
#                 forwards_rec = forwards_rec.drop(next_player.name)
#                 print(f"Added {next_player['name']} ({next_player['position']}) - ${next_player['salary']:,}")
#                 print(f"Remaining Salary Cap: ${remaining_cap:,}")
#             else:
#                 print(f"No suitable player found for position {position} within the remaining salary cap.")
#                 break
#         elif position == 'D':
#             next_player, remaining_cap = recommend_next_player_skater(selected_player, defense_rec, remaining_cap, performance_metrics)
#             if next_player is not None:
#                 recommended_team.append(next_player)
#                 defense_rec = defense_rec.drop(next_player.name)
#                 print(f"Added {next_player['name']} ({next_player['position']}) - ${next_player['salary']:,}")
#                 print(f"Remaining Salary Cap: ${remaining_cap:,}")
#             else:
#                 print(f"No suitable player found for position {position} within the remaining salary cap.")
#                 break
        
#         player_count += 1
#         # Check if the remaining salary cap is at least $1,000,000 after the fourth selection
#         if player_count == 4 and remaining_cap < 1000000:
#             print("Remaining salary cap is less than $1,000,000. Adjusting selection process to ensure the last pick can be made.")
#             # Remove the last selected player and adjust the selection process
#             last_player = recommended_team.pop()
#             remaining_cap += last_player['salary']
#             if last_player['position'] == 'F':
#                 forwards_rec = pd.concat([forwards_rec, pd.DataFrame([last_player])], ignore_index=True)
#             elif last_player['position'] == 'D':
#                 defense_rec = pd.concat([defense_rec, pd.DataFrame([last_player])], ignore_index=True)
#             # Adjust the selection process to ensure the last pick can be made
#             if position == 'F':
#                 forwards_rec = forwards_rec[forwards_rec['salary'] <= remaining_cap]
#             elif position == 'D':
#                 defense_rec = defense_rec[defense_rec['salary'] <= remaining_cap]

# # Ensure the team composition requirements are met
# while len([p for p in recommended_team if p['position'] == 'F']) < team_composition['F']:
#     next_player, remaining_cap = recommend_next_player_skater(selected_player, forwards_rec, remaining_cap, performance_metrics)
#     if next_player is not None:
#         recommended_team.append(next_player)
#         forwards_rec = forwards_rec.drop(next_player.name)
#         print(f"Added {next_player['name']} ({next_player['position']}) - ${next_player['salary']:,}")
#         print(f"Remaining Salary Cap: ${remaining_cap:,}")
#     else:
#         print(f"No suitable player found for position F within the remaining salary cap.")
#         break

# while len([p for p in recommended_team if p['position'] == 'D']) < team_composition['D']:
#     next_player, remaining_cap = recommend_next_player_skater(selected_player, defense_rec, remaining_cap, performance_metrics)
#     if next_player is not None:
#         recommended_team.append(next_player)
#         defense_rec = defense_rec.drop(next_player.name)
#         print(f"Added {next_player['name']} ({next_player['position']}) - ${next_player['salary']:,}")
#         print(f"Remaining Salary Cap: ${remaining_cap:,}")
#     else:
#         print(f"No suitable player found for position D within the remaining salary cap.")
#         break

# # Final check to ensure the team composition requirements are met
# if len([p for p in recommended_team if p['position'] == 'F']) == team_composition['F'] and len([p for p in recommended_team if p['position'] == 'D']) == team_composition['D']:
#     # Convert recommended team to a DataFrame to view/inspect the team
#     recommended_team_df = pd.DataFrame(recommended_team)
#     print("Recommended Team:")
#     print(recommended_team_df[['name', 'position', 'salary', 'points', 'goals', 'assists']])
#     print(f"Remaining Salary Cap: ${remaining_cap:,}")
# else:
#     print("Failed to meet the team composition requirements.")

In [1726]:
# # Initialize the team with the selected player
# recommended_team = [selected_player]

# # For randomness, shuffle the positions list.
# positions = ['F', 'D']
# random.shuffle(positions)

# # Now, we pick players for each position.
# # We'll loop over the positions and within each add the required number of players.
# for position in positions:
#     count = team_composition[position]
#     for _ in range(count):
#         # Determine "reserve" amount: if this is not the final pick, leave at least $1,000,000.
#         if len(recommended_team) < desired_team_count - 1:
#             reserve = 1000000
#         else:
#             reserve = 0
        
#         if position == 'F':
#             next_player, remaining_cap = recommend_next_player_skater(selected_player, forwards_rec, remaining_cap, performance_metrics, reserve)
#             if next_player is not None:
#                 recommended_team.append(next_player)
#                 forwards_rec = forwards_rec.drop(next_player.name)
#                 print(f"Added {next_player['name']} ({next_player['position']}) - ${next_player['salary']:,}")
#                 print(f"Remaining Salary Cap: ${remaining_cap:,}")
#             else:
#                 print(f"No suitable forward found with at least $1,000,000 reserve remaining.")
#                 break
#         elif position == 'D':
#             next_player, remaining_cap = recommend_next_player_skater(selected_player, defense_rec, remaining_cap, performance_metrics, reserve)
#             if next_player is not None:
#                 recommended_team.append(next_player)
#                 defense_rec = defense_rec.drop(next_player.name)
#                 print(f"Added {next_player['name']} ({next_player['position']}) - ${next_player['salary']:,}")
#                 print(f"Remaining Salary Cap: ${remaining_cap:,}")
#             else:
#                 print(f"No suitable defense found with at least $1,000,000 reserve remaining.")
#                 break

# # Final check: must have exactly 5 players (3 F, 2 D)
# if (len([p for p in recommended_team if p['position'] == 'F']) == team_composition['F'] and
#     len([p for p in recommended_team if p['position'] == 'D']) == team_composition['D']):
    
#     recommended_team_df = pd.DataFrame(recommended_team)
#     print("Recommended Team:")
#     print(recommended_team_df[['name', 'position', 'salary', 'points', 'goals', 'assists']])
#     print(f"Remaining Salary Cap: ${remaining_cap:,}")
# else:
#     print("Failed to meet the team composition requirements.")

In [1727]:
# # Initialize recommended team with the selected player
# recommended_team = [selected_player]

# # We will now pick players. For simplicity, we loop over each required pos.
# # In each pick (except the final one) we require a "reserve" of $1,000,000.
# for pos, count in team_composition.items():
#     for i in range(count):
#         # Determine reserve: if this is not the final pick overall, reserve $1,000,000.
#         if len(recommended_team) < desired_team_count - 1:
#             reserve = 1000000
#         else:
#             reserve = 0
        
#         if pos == 'F':
#             candidate, new_cap = recommend_next_player_skater(selected_player, forwards_rec, remaining_cap, performance_metrics, reserve)
#             if candidate is None:
#                 print(f"No suitable forward candidate found with at least $1,000,000 reserved.")
#                 continue
#             recommended_team.append(candidate)
#             forwards_rec = forwards_rec.drop(candidate.name)
#             remaining_cap = new_cap
#             print(f"Added {candidate['name']} (F) - ${candidate['salary']:,}")
#             print(f"Remaining Salary Cap: ${remaining_cap:,}")
#         elif pos == 'D':
#             candidate, new_cap = recommend_next_player_skater(selected_player, defense_rec, remaining_cap, performance_metrics, reserve)
#             if candidate is None:
#                 print(f"No suitable defense candidate found with at least $1,000,000 reserved.")
#                 continue
#             recommended_team.append(candidate)
#             defense_rec = defense_rec.drop(candidate.name)
#             remaining_cap = new_cap
#             print(f"Added {candidate['name']} (D) - ${candidate['salary']:,}")
#             print(f"Remaining Salary Cap: ${remaining_cap:,}")
#         player_count += 1
#         # Check if the remaining salary cap is at least $1,000,000 after the fourth selection
#         if player_count == 4 and remaining_cap < 1000000:
#             print("Remaining salary cap is less than $1,000,000. Adjusting selection process to ensure the last pick can be made.")
#             # Remove the last selected player and adjust the selection process
#             last_player = recommended_team.pop()
#             remaining_cap += last_player['salary']
#             if last_player['pos'] == 'F':
#                 forwards_rec = pd.concat([forwards_rec, pd.DataFrame([last_player])], ignore_index=True)
#             elif last_player['pos'] == 'D':
#                 defense_rec = pd.concat([defense_rec, pd.DataFrame([last_player])], ignore_index=True)
#             # Adjust the selection process to ensure the last pick can be made
#             if pos == 'F':
#                 forwards_rec = forwards_rec[forwards_rec['salary'] <= remaining_cap]
#             elif pos == 'D':
#                 defense_rec = defense_rec[defense_rec['salary'] <= remaining_cap]
# # Final check: must have exactly 5 players (3 F, 2 D)
# if (len([p for p in recommended_team if p['position'] == 'F']) == team_composition['F'] and
#     len([p for p in recommended_team if p['position'] == 'D']) == team_composition['D']):
#     # Convert recommended team to a DataFrame to view/inspect the team
#     recommended_team_df = pd.DataFrame(recommended_team)


# print("Recommended Team:")
# print(recommended_team_df[['name', 'position', 'salary', 'points', 'goals', 'assists']])
# print(f"Remaining Salary Cap: ${remaining_cap:,}")

In [1728]:
# # Initialize the team with the selected player
# recommended_team = [selected_player]

# # Shuffle the positions to pick players from different positions in a random order
# positions = ['F', 'D']
# random.shuffle(positions)

# # Recommend players for each position
# player_count = 1  # Start with the first selected player
# for position in positions:
#     count = team_composition[position]
#     for _ in range(count):
#         if position == 'F':
#             next_player, remaining_cap = recommend_next_player_skater(selected_player, forwards_rec, remaining_cap, performance_metrics)
#             if next_player is not None:
#                 recommended_team.append(next_player)
#                 forwards_rec = forwards_rec.drop(next_player.name)
#                 print(f"Added {next_player['name']} ({next_player['position']}) - ${next_player['salary']:,}")
#                 print(f"Remaining Salary Cap: ${remaining_cap:,}")
#             else:
#                 print(f"No suitable player found for position {position} within the remaining salary cap.")
#                 break
#         elif position == 'D':
#             next_player, remaining_cap = recommend_next_player_skater(selected_player, defense_rec, remaining_cap, performance_metrics)
#             if next_player is not None:
#                 recommended_team.append(next_player)
#                 defense_rec = defense_rec.drop(next_player.name)
#                 print(f"Added {next_player['name']} ({next_player['position']}) - ${next_player['salary']:,}")
#                 print(f"Remaining Salary Cap: ${remaining_cap:,}")
#             else:
#                 print(f"No suitable player found for position {position} within the remaining salary cap.")
#                 break
        
#         player_count += 1
#         # Check if the remaining salary cap is at least $1,000,000 after the fourth selection
#         if player_count == 4 and remaining_cap < 1000000:
#             print("Remaining salary cap is less than $1,000,000. Adjusting selection process to ensure the last pick can be made.")
#             # Remove the last selected player and adjust the selection process
#             last_player = recommended_team.pop()
#             remaining_cap += last_player['salary']
#             if last_player['position'] == 'F':
#                 forwards_rec = pd.concat([forwards_rec, pd.DataFrame([last_player])], ignore_index=True)
#             elif last_player['position'] == 'D':
#                 defense_rec = pd.concat([defense_rec, pd.DataFrame([last_player])], ignore_index=True)
#             # Adjust the selection process to ensure the last pick can be made
#             if position == 'F':
#                 forwards_rec = forwards_rec[forwards_rec['salary'] <= remaining_cap]
#             elif position == 'D':
#                 defense_rec = defense_rec[defense_rec['salary'] <= remaining_cap]

# # Ensure the team composition requirements are met
# while len([p for p in recommended_team if p['position'] == 'F']) < team_composition['F']:
#     next_player, remaining_cap = recommend_next_player_skater(selected_player, forwards_rec, remaining_cap, performance_metrics)
#     if next_player is not None:
#         recommended_team.append(next_player)
#         forwards_rec = forwards_rec.drop(next_player.name)
#         print(f"Added {next_player['name']} ({next_player['position']}) - ${next_player['salary']:,}")
#         print(f"Remaining Salary Cap: ${remaining_cap:,}")
#     else:
#         print(f"No suitable player found for position F within the remaining salary cap.")
#         break

# while len([p for p in recommended_team if p['position'] == 'D']) < team_composition['D']:
#     next_player, remaining_cap = recommend_next_player_skater(selected_player, defense_rec, remaining_cap, performance_metrics)
#     if next_player is not None:
#         recommended_team.append(next_player)
#         defense_rec = defense_rec.drop(next_player.name)
#         print(f"Added {next_player['name']} ({next_player['position']}) - ${next_player['salary']:,}")
#         print(f"Remaining Salary Cap: ${remaining_cap:,}")
#     else:
#         print(f"No suitable player found for position D within the remaining salary cap.")
#         break

# # Convert recommended team to a DataFrame to view/inspect the team
# recommended_team_df = pd.DataFrame(recommended_team)

# print("Recommended Team:")
# print(recommended_team_df[['name', 'position', 'salary', 'points', 'goals', 'assists']])
# print(f"Remaining Salary Cap: ${remaining_cap:,}")